<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D8%B3%D8%AA%D8%B1%D8%A7%D8%AA%D9%8A%D8%AC%D9%8A%D8%A9_%D9%83%D8%A7%D9%85%D9%84%D8%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 1.5 * atr_value if order_type == "buy" else price + 1.5 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        print(f"RSI: {data['RSI'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        print(f"Momentum: {data['Momentum'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        if 'volume' in data.columns:
            data['Volume_MA'] = data['volume'].rolling(window=20).mean()
            print(f"Volume Moving Average: {data['Volume_MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        atr = calculate_atr(data)
        data['ATR'] = atr
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)
        print(f"ATR: {data['ATR'].iloc[-1]}, Stop Loss: {data['Stop_Loss'].iloc[-1]}, Take Profit: {data['Take_Profit'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_momentum(data)
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # إنشاء ميزات إضافية
        data = create_features(data)
        data = calculate_price_channel(data)
        data = advanced_risk_management(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على المؤشرات
        last_row = data.iloc[-1]
        print(f"MA: {last_row['MA']}, Resistance: {last_row['Resistance']}, RSI: {last_row['RSI']}, Support: {last_row['Support']}")

        if not check_open_positions("XAUUSD"):  # تحقق من عدم وجود صفقات مفتوحة
            if (last_row['MA'] > last_row['Resistance']) and (last_row['RSI'] < 70):
                place_order("XAUUSD", "buy", volume=0.01)
            elif (last_row['MA'] < last_row['Support']) and (last_row['RSI'] > 30):
                place_order("XAUUSD", "sell", volume=0.01)

        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)
        time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار



In [ ]:

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 1.5 * atr_value if order_type == "buy" else price + 1.5 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        print(f"RSI: {data['RSI'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        print(f"Momentum: {data['Momentum'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        if 'volume' in data.columns:
            data['Volume_MA'] = data['volume'].rolling(window=20).mean()
            print(f"Volume Moving Average: {data['Volume_MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        atr = calculate_atr(data)
        data['ATR'] = atr
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)
        print(f"ATR: {data['ATR'].iloc[-1]}, Stop Loss: {data['Stop_Loss'].iloc[-1]}, Take Profit: {data['Take_Profit'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_momentum(data)
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        wolf_wave_points = calculate_wolf_wave(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and not check_open_positions("XAUUSD"):
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and not check_open_positions("XAUUSD"):
            place_order("XAUUSD", "sell", volume=0.01)

        # إنشاء ميزات إضافية
        data = create_features(data)
        data = calculate_price_channel(data)
        data = advanced_risk_management(data)

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if not check_open_positions("XAUUSD"):  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)
        time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار
#و الحمد لله رب العالمين


In [ ]:
و الحمد لله رب العالمين

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

def hedge_position(symbol, volume):
    """فتح صفقة معاكسة لتأمين الصفقة الحالية."""
    try:
        if not check_open_positions(symbol):
            print(f"No open positions to hedge for {symbol}.")
            return

        current_price = mt5.symbol_info_tick(symbol).bid
        order_type = "sell" if mt5.positions_get(symbol=symbol)[0].type == 0 else "buy"
        opposite_order_type = "buy" if order_type == "sell" else "sell"

        # فتح صفقة معاكسة
        place_order(symbol, opposite_order_type, volume)
        print(f"Hedged position by placing a {opposite_order_type} order for {volume} lots of {symbol}.")

    except Exception as e:
        print(f"An error occurred while hedging position: {e}")


def set_break_even(symbol):
    """تعديل وقف الخسارة إلى سعر الدخول إذا كانت الصفقة في الربح."""
    try:
        positions = mt5.positions_get(symbol=symbol)
        for position in positions:
            if position.type == 0:  # إذا كانت الصفقة شراء
                price = position.price_open
                if position.profit > 10:  # إذا كانت الربح أكبر من 10
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": position.ticket,
                        "sl": price,  # تعديل SL إلى سعر الدخول
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"Break even set for {symbol} at price {price}.")
                    else:
                        print(f"Failed to set break even for {symbol}: {result.retcode}")

            elif position.type == 1:  # إذا كانت الصفقة بيع
                price = position.price_open
                if position.profit > 10:  # إذا كانت الربح أكبر من 10
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": position.ticket,
                        "sl": price,  # تعديل SL إلى سعر الدخول
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"Break even set for {symbol} at price {price}.")
                    else:
                        print(f"Failed to set break even for {symbol}: {result.retcode}")

    except Exception as e:
        print(f"An error occurred while setting break even: {e}")





def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 1.5 * atr_value if order_type == "buy" else price + 1.5 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        print(f"RSI: {data['RSI'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        print(f"Momentum: {data['Momentum'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        if 'volume' in data.columns:
            data['Volume_MA'] = data['volume'].rolling(window=20).mean()
            print(f"Volume Moving Average: {data['Volume_MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        atr = calculate_atr(data)
        data['ATR'] = atr
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)
        print(f"ATR: {data['ATR'].iloc[-1]}, Stop Loss: {data['Stop_Loss'].iloc[-1]}, Take Profit: {data['Take_Profit'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_momentum(data)
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value


        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position("XAUUSD", volume=0.01)
            set_break_even("XAUUSD")  # تعيين بريك إيفن




        # إنشاء ميزات إضافية
        data = create_features(data)
        data = calculate_price_channel(data)
        data = advanced_risk_management(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على المؤشرات
        last_row = data.iloc[-1]
        print(f"MA: {last_row['MA']}, Resistance: {last_row['Resistance']}, RSI: {last_row['RSI']}, Support: {last_row['Support']}")

        if not check_open_positions("XAUUSD"):  # تحقق من عدم وجود صفقات مفتوحة
            if (last_row['MA'] > last_row['Resistance']) and (last_row['RSI'] < 70):
                place_order("XAUUSD", "buy", volume=0.01)
            elif (last_row['MA'] < last_row['Support']) and (last_row['RSI'] > 30):
                place_order("XAUUSD", "sell", volume=0.01)

        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)
        time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار

